In [ ]:
import json
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import os

# Option to specify UUIDs to display
uuids_to_display = None  # default - random
uuids_to_display = ['6005aa68-9832-3c24-7bd1-d1bdfd001a33', 'def29391-2b9b-20e1-1e7c-ac37d525729f', 'b8b9b548-b14b-cca5-55ce-e6edecbc0ee2', 'c1d8e07e-a336-d55e-1a75-92b075b2f912', '73928170-7fbd-99c3-8fe7-aaa7bb6b18c9']

# Load the JSON file
json_path = '/fs/ess/PAS2136/ggr_data/results/GGR2020_subset_refactor_final/detector/img_annots_320.json'
with open(json_path, 'r') as f:
    data = json.load(f)

print(f"Total images: {len(data['images'])}")
print(f"Total annotations: {len(data['annotations'])}")

# Create a mapping from image_uuid to image info
image_map = {img['uuid']: img for img in data['images']}

# Group annotations by image
annotations_by_image = {}
for ann in data['annotations']:
    img_uuid = ann['image_uuid']
    if img_uuid not in annotations_by_image:
        annotations_by_image[img_uuid] = []
    annotations_by_image[img_uuid].append(ann)

# Get images that have annotations
images_with_annotations = list(annotations_by_image.keys())
print(f"Images with detections: {len(images_with_annotations)}")

# Select images to display
if uuids_to_display is None:
    # Select 5 random images with annotations
    selected_images = random.sample(images_with_annotations, min(5, len(images_with_annotations)))
else:
    # Use specified UUIDs (filter to only those that exist and have annotations)
    selected_images = [uuid for uuid in uuids_to_display if uuid in images_with_annotations]
    if len(selected_images) < len(uuids_to_display):
        print(f"Warning: Some UUIDs not found or have no annotations")

# Print selected UUIDs for future use
print(f"\nSelected {len(selected_images)} images")
print("\nTo re-display these same images, copy and paste this line at the beginning:")
print(f"uuids_to_display = {selected_images}")

# Visualize the detections
fig, axes = plt.subplots(1, len(selected_images), figsize=(4*len(selected_images), 4))
if len(selected_images) == 1:
    axes = [axes]  # Make it a list for consistency

for idx, img_uuid in enumerate(selected_images):
    # Get image info and annotations
    img_info = image_map[img_uuid]
    img_path = img_info['image_path']
    annotations = annotations_by_image[img_uuid]
    
    # Check if image exists
    if not os.path.exists(img_path):
        print(f"Warning: Image not found at {img_path}")
        axes[idx].text(0.5, 0.5, 'Image not found', 
                      ha='center', va='center', transform=axes[idx].transAxes)
        axes[idx].set_title(f"Image {idx+1}")
        continue
    
    # Load and display image
    img = Image.open(img_path)
    axes[idx].imshow(img)
    
    # Draw bounding boxes
    for ann in annotations:
        # bbox format: [x, y, width, height]
        x, y, w, h = ann['bbox']
        rect = patches.Rectangle((x, y), w, h, linewidth=2, 
                               edgecolor='red', facecolor='none')
        axes[idx].add_patch(rect)
        
        # Add confidence score
        conf_text = f"{ann['confidence']:.2f}"
        axes[idx].text(x, y-5, conf_text, color='red', 
                      fontsize=8, weight='bold',
                      bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    axes[idx].set_title(f"Image {idx+1}: {len(annotations)} detections")
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# Print summary of the visualized images
print("\nSummary of visualized images:")
for idx, img_uuid in enumerate(selected_images):
    img_info = image_map[img_uuid]
    annotations = annotations_by_image[img_uuid]
    print(f"\nImage {idx+1}:")
    print(f"  UUID: {img_uuid}")
    print(f"  Path: {os.path.basename(img_info['image_path'])}")
    print(f"  Detections: {len(annotations)}")
    for ann in annotations[:3]:  # Show first 3 detections
        print(f"    - Class {ann['detection_class']}, Confidence: {ann['confidence']:.3f}")